In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
df = pd.read_csv("age_gender.csv")
df.head()

,age,ethnicity,gender,img_name,pixels
0,1,2,0,20161219203650636.jpg.chip.jpg,129 128 128 126 127 130 133 135 139 142 145 14...
1,1,2,0,20161219222752047.jpg.chip.jpg,164 74 111 168 169 171 175 182 184 188 193 199...
2,1,2,0,20161219222832191.jpg.chip.jpg,67 70 71 70 69 67 70 79 90 103 116 132 145 155...
3,1,2,0,20161220144911423.jpg.chip.jpg,193 197 198 200 199 200 202 203 204 205 208 21...
4,1,2,0,20161220144914327.jpg.chip.jpg,202 205 209 210 209 209 210 211 212 214 218 21...


In [4]:
#Checking for NA values
df.isna().sum()

age          0
ethnicity    0
gender       0
img_name     0
pixels       0
dtype: int64

In [5]:
Y = df['ethnicity']
Y.nunique()

5

In [6]:
#Getting dimensions of pixels
n_features = len(df.pixels[0].split())
print(len(df.pixels), n_features)


23705 2304


In [7]:
#Spliting String into respective floats
X=np.zeros(shape=(23705,2304))

for i in range(len(df.pixels)):
    a=np.array(df.pixels[i].split(),dtype='float32')
    X[i]=a

In [8]:
X

array([[129., 128., 128., ..., 146., 146., 146.],
       [164.,  74., 111., ..., 182., 170., 148.],
       [ 67.,  70.,  71., ..., 112., 111., 108.],
       ...,
       [ 59.,  50.,  37., ...,  98.,  78.,  78.],
       [ 45., 108., 120., ...,  32.,  35.,  35.],
       [156., 161., 160., ..., 190., 184., 174.]])

In [9]:
X = X.reshape(-1,48,48)

In [10]:
X.shape

(23705, 48, 48)

In [11]:
n = len(X)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)

In [13]:
class ImageDataTrain(Dataset):
    def __init__(self):
        self.x = torch.unsqueeze(torch.from_numpy(X_train).float(),1)
        self.y = torch.unsqueeze(torch.Tensor(y_train.values).float(),1)
        self.n_samples = len(y_train)
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.n_samples

dataset_tr = ImageDataTrain()    
train_loader = DataLoader(dataset=dataset_tr, batch_size=50, shuffle=True, num_workers=0)

train_iter = iter(train_loader)
train_data = train_iter.next()

In [14]:
class ImageDataTest(Dataset):
    def __init__(self):
        self.x = torch.unsqueeze(torch.from_numpy(X_test).float(),1)
        self.y = torch.unsqueeze(torch.Tensor(y_test.values).float(),1)
        self.n_samples = len(y_test)
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.n_samples

dataset_te = ImageDataTest()    
test_loader = DataLoader(dataset=dataset_te, batch_size=500, shuffle=True, num_workers=0)

test_iter = iter(test_loader)
test_data = test_iter.next()
test_features, test_labels = test_data

In [15]:
class Ethinicity(nn.Module):
    def __init__(self):
        super(Ethinicity,self).__init__()
        
        self.conv_layers = nn.Sequential(
                        #Conv Layer 1
                        nn.Conv2d(in_channels=1,out_channels=32, kernel_size=3,stride=1,padding=1),
                        nn.ReLU(inplace=True),
                        nn.BatchNorm2d(32),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        #Conv Layer 2
                        nn.Conv2d(in_channels=32,out_channels=64, kernel_size=3,stride=1,padding=1),
                        nn.ReLU(inplace=True),
                        nn.BatchNorm2d(64),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        #Conv Layer 3
                        nn.Conv2d(in_channels=64,out_channels=128, kernel_size=3,stride=1,padding=1),
                        nn.ReLU(inplace=True),
                        nn.BatchNorm2d(128),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.linear_layers = nn.Sequential(
                        nn.Linear(128*6*6,5))
        
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0),-1)
        x = self.linear_layers(x)
        return x

net = Ethinicity()
print(net)

Ethinicity(
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=4608, out_features=5, bias=True)
  )
)


In [16]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.003)


In [17]:
num_epochs = 5

In [18]:
total_step = len(train_loader)
net.train()
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
                
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels.squeeze().long())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/380], Loss: 1.7809
Epoch [1/5], Step [200/380], Loss: 1.2058
Epoch [1/5], Step [300/380], Loss: 0.8770
Epoch [2/5], Step [100/380], Loss: 0.5989
Epoch [2/5], Step [200/380], Loss: 0.7937
Epoch [2/5], Step [300/380], Loss: 0.9799
Epoch [3/5], Step [100/380], Loss: 0.6487
Epoch [3/5], Step [200/380], Loss: 0.6695
Epoch [3/5], Step [300/380], Loss: 0.6909
Epoch [4/5], Step [100/380], Loss: 0.4725
Epoch [4/5], Step [200/380], Loss: 0.9247
Epoch [4/5], Step [300/380], Loss: 0.5826
Epoch [5/5], Step [100/380], Loss: 0.3374
Epoch [5/5], Step [200/380], Loss: 0.5193
Epoch [5/5], Step [300/380], Loss: 0.4073


In [19]:
net.eval()
pred_labels = net(test_features)

In [20]:
a = pred_labels.detach().numpy().tolist()
y_pred = [int(a[i].index(max(a[i]))) for i in range(len(a))]
a = test_labels.numpy().tolist()
y_test = [int(a[i][0]) for i in range(len(a))]

In [21]:
correct = 0
incorrect = 0
for i in range(len(y_test)):
    if y_pred[i] == y_test[i]:
        correct += 1
    else:
        incorrect += 1


In [22]:
print("Accuracy :",correct/(correct+incorrect)*100, "%")

Accuracy : 75.8 %


In [23]:
# Save the model checkpoint
torch.save(net.state_dict(), 'model.ckpt')